<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Откроем-и-изучим-данные" data-toc-modified-id="Откроем-и-изучим-данные-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Откроем и изучим данные</a></span><ul class="toc-item"><li><span><a href="#Откроем-данные" data-toc-modified-id="Откроем-данные-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Откроем данные</a></span></li><li><span><a href="#Изучим-данные" data-toc-modified-id="Изучим-данные-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Изучим данные</a></span></li></ul></li><li><span><a href="#Проведем-предобработку-данных" data-toc-modified-id="Проведем-предобработку-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Проведем предобработку данных</a></span><ul class="toc-item"><li><span><a href="#Проведем-предобработку-df_xls" data-toc-modified-id="Проведем-предобработку-df_xls-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Проведем предобработку df_xls</a></span></li><li><span><a href="#Проведем-предобраобтку-df_subtitles" data-toc-modified-id="Проведем-предобраобтку-df_subtitles-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Проведем предобраобтку df_subtitles</a></span></li><li><span><a href="#Проведем-предобраобтку-df_dictionaries" data-toc-modified-id="Проведем-предобраобтку-df_dictionaries-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Проведем предобраобтку df_dictionaries</a></span></li><li><span><a href="#Объединим-таблицы" data-toc-modified-id="Объединим-таблицы-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Объединим таблицы</a></span></li></ul></li><li><span><a href="#Обучим-модель" data-toc-modified-id="Обучим-модель-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучим модель</a></span></li></ul></div>

# Мастерская 2 #

Проект выполняется в рамках Мастерской Яндекс Практикума.
Выполнила: Дмитрак Наталья @n_dmitrak

**Цель** — создание системы для автоматического определения уровня сложности англоязычных фильмов.<br>


**Использованные данные:**<br>
    
-субтитры к фильмам<br>
-словари для определения уровня знания английского языка<br> 

**Задача**<br>
Разработка ML-решения для автоматического определения уровня сложности англоязычных фильмов.<br>    
Важно выбрать фильм, который подходит студенту по уровню сложности, т.е. студент должен понимать 50-70% диалогов.<br>
Необходимо обучить языковую модель, разработать для неё веб-интерфейс и создать микросервис.   
    
**Ход реализации**<br>
1. Загрузим данные<br>
2. Проведем предварительную обработку данных/EDA<br>  
3. Обучим модель
4. Оценим качество получившейся модели

## Откроем и изучим данные ##

In [ ]:
# загрузим необходимые библиотеки 

import pandas as pd
import glob
import os
import re
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import RandomOverSampler
import pdfplumber
import pysrt
import chardet
import sweetviz as sv
import matplotlib.pyplot as plt

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

### Откроем данные ### 

In [2]:
# загрузим таблицу 

xls_file_path = '/Users/nata/Desktop/Мастерская-2/English_level/English_scores/movies_labels.xlsx'
try:
    df_xls = pd.read_excel(xls_file_path)
    print("Таблица xls загружена с компьютера")
except FileNotFoundError:
    print("Файл xls не найден на компьютере")
    # если загрузка с компьютера не удалась, загрузим таблицу xls из интернета
    xls_url = 'https://docviewer.yandex.ru/view/786181067/?*=exuH9EgdsE1zPMskziJKflErhIF7InVybCI6InlhLWRpc2stcHVibGljOi8vSjlIK09zQm9ralMwblM2L1ZvUVNURDFHMGg4YTlTUitzMWg1bDdSU1RkQ2xwYnhtZnRwd01EdGRXRFBJcjJORnEvSjZicG1SeU9Kb25UM1ZvWG5EYWc9PTovRW5nbGlzaF9zY29yZXMuemlwIiwidGl0bGUiOiJFbmdsaXNoX3Njb3Jlcy56aXAiLCJub2lmcmFtZSI6ZmFsc2UsInVpZCI6Ijc4NjE4MTA2NyIsInRzIjoxNjg4ODM1MjEzNDE1LCJ5dSI6IjEzNTI1MzgyMjE2Nzg0MzUxOTgifQ%3D%3D'
    try:
        response = requests.get(xls_url)
        df_xls = pd.read_excel(BytesIO(response.content))
        print("Таблица xls загружена из интернета")
    except Exception as e:
        print("Ошибка при загрузке таблицы xls:", e)

# выведем загруженную таблицу
print("Таблица xls:")
print(df_xls.head())

Таблица xls загружена с компьютера
Таблица xls:
   id                             Movie   Level
0   0         10_Cloverfield_lane(2016)      B1
1   1  10_things_I_hate_about_you(1999)      B1
2   2              A_knights_tale(2001)      B2
3   3              A_star_is_born(2018)      B2
4   4                     Aladdin(1992)  A2/A2+


In [3]:
# загрузим субтитры 

subtitles_list = []

# загрузим субтитры из локальных файлов
for dirpath, dirs, files in os.walk('/Users/nata/Desktop/Мастерская-2'):
    for file in files:
        if file.endswith('.srt'):
            # определим кодировку файла
            rawdata = open(os.path.join(dirpath, file), 'rb').read()
            result = chardet.detect(rawdata)
            encoding = result['encoding']
            
            # откроем файл с обнаруженной кодировкой
            with open(os.path.join(dirpath, file), 'r', encoding=encoding, errors='ignore') as f:
                text = f.read()
                movie_name = file[:-4]  # извлечем название фильма из имени файла
                subtitles_list.append((movie_name, text))

# загрузим субтитры из интернета
urls = [
    'https://docviewer.yandex.ru/view/786181067/?*=Uc72FZjr7UQ%2FQx1Vjre7%2B1U4JBh7InVybCI6InlhLWRpc2stcHVibGljOi8vSjlIK09zQm9ralMwblM2L1ZvUVNURDFHMGg4YTlTUitzMWg1bDdSU1RkQ2xwYnhtZnRwd01EdGRXRFBJcjJORnEvSjZicG1SeU9Kb25UM1ZvWG5EYWc9PTovRW5nbGlzaF9zY29yZXMuemlwIiwidGl0bGUiOiJFbmdsaXNoX3Njb3Jlcy56aXAiLCJub2lmcmFtZSI6ZmFsc2UsInVpZCI6Ijc4NjE4MTA2NyIsInRzIjoxNjg4OTMyNDcxMjcxLCJ5dSI6IjEzNTI1MzgyMjE2Nzg0MzUxOTgifQ%3D%3D',
    'https://disk.yandex.ru/d/CkmiSRnKba76sA'
]

for url in urls:
    try:
        response = requests.get(url)
        df_temp = pd.read_csv(BytesIO(response.content), sep='\t', header=None)
        movie_name = df_temp.iloc[0, 0]
        subtitles = df_temp.iloc[1:, 0].str.cat(sep='\n')
        subtitles_list.append((movie_name, subtitles))
        print(f"Субтитры загружены: {movie_name}")
    except Exception as e:
        print("Ошибка при загрузке субтитров:", e)

# создадим DataFrame субтитров
df_subtitles = pd.DataFrame(subtitles_list, columns=['movie', 'subtitles'])

# напечатаем загруженную таблицу
print("Таблица субтитров:")
print(df_subtitles.head())

Ошибка при загрузке субтитров: name 'requests' is not defined
Ошибка при загрузке субтитров: name 'requests' is not defined
Таблица субтитров:
                                               movie  \
0            Crown, The S01E01 - Wolferton Splash.en   
1                            Suits.Episode 1- Denial   
2  Crazy4TV.com - Suits.S06E06.720p.BluRay.x265.H...   
3                      Suits.S02E08.HDTV.x264-EVOLVE   
4  Virgin.River.S01E07.INTERNAL.720p.WEB.x264-STRiFE   

                                           subtitles  
0  1\n00:00:59,400 --> 00:01:03,960\nIn seeking h...  
1  1\n00:00:07,298 --> 00:00:09,332\nYou're the m...  
2  1\n00:00:00,877 --> 00:00:02,337\n(HARVEY READ...  
3  1\n00:00:05,640 --> 00:00:07,641\n[Car horn bl...  
4  1\n00:00:10,468 --> 00:00:13,178\nAre you sure...  


In [4]:
# загрузим словари 

# создадим пустой DataFrame для хранения данных
df_dictionaries = pd.DataFrame(columns=['word', 'level'])

current_level = None

# загрузим словари из локальных файлов
for dict_file in os.listdir('/Users/nata/Desktop/Мастерская-2/English_level/Oxford_CEFR_level'):
    if dict_file.endswith('.pdf'):
        with pdfplumber.open(os.path.join('/Users/nata/Desktop/Мастерская-2/English_level/Oxford_CEFR_level', dict_file)) as pdf:
            for page in pdf.pages:
                text = page.extract_text()
                for line in text.split('\n'):
                    if line.strip():  # проверим, что строка не пустая
                        line = line.split()  # разделим строку на слова
                        if len(line) == 1 and line[0] in ['A1', 'A2', 'B1', 'B2', 'C1', 'C2']:  # проверим, является ли это новым уровнем
                            current_level = line[0]
                        elif current_level is not None:
                            for word in line:
                                if word.isalpha():  # проверим, состоит ли слово только из букв
                                    new_row = pd.DataFrame({'word': [word], 'level': [current_level]})
                                    df_dictionaries = pd.concat([df_dictionaries, new_row], ignore_index=True)

# загрузим словари из интернета
urls = [
    'https://docviewer.yandex.ru/view/786181067/?*=4L9A%2FdYJlUiDJaD0TeyudQgZZON7InVybCI6InlhLWRpc2stcHVibGljOi8vSjlIK09zQm9ralMwblM2L1ZvUVNURDFHMGg4YTlTUitzMWg1bDdSU1RkQ2xwYnhtZnRwd01EdGRXRFBJcjJORnEvSjZicG1SeU9Kb25UM1ZvWG5EYWc9PTovT3hmb3JkX0NFRlJfbGV2ZWwuemlwIiwidGl0bGUiOiJPeGZvcmRfQ0VGUl9sZXZlbC56aXAiLCJub2lmcmFtZSI6ZmFsc2UsInVpZCI6Ijc4NjE4MTA2NyIsInRzIjoxNjg4OTMyNjAwMzE2LCJ5dSI6IjEzNTI1MzgyMjE2Nzg0MzUxOTgifQ%3D%3D'
]

for url in urls:
    try:
        response = requests.get(url)
        df_temp = pd.read_csv(BytesIO(response.content), sep='\t', header=None)
        df_temp.columns = ['word', 'level']
        df_dictionaries = pd.concat([df_dictionaries, df_temp], ignore_index=True)
        print("Словарь загружен")
    except Exception as e:
        print("Ошибка при загрузке словаря:", e)

# напечатаем получившийся DataFrame
print("Таблица словарей:")
print(df_dictionaries.head())

Ошибка при загрузке словаря: name 'requests' is not defined
Таблица словарей:
     word level
0    bank    A1
1     buy    A1
2  course    A1
3     bar    A1
4      by    A1


### Изучим данные ### 

In [5]:
# создадим функцию для получения информации о данных

def get_data_info(data):
    print(data)
    print("Общая информация:")
    display(data.info())

    print("Первые 5 строк:")
    display(data.head())

    print("Количество уникальных значений в каждом столбце:")
    for column in data.columns:
        if not isinstance(data[column].iloc[0], list):
            print(f"Столбец: {column}")
            print(f"Количество уникальных значений: {data[column].nunique()}")
            print("--------------------")

In [6]:
# создадим словарь с названиями датафреймов
dfs = {'xls': df_xls, 'subtitles': df_subtitles, 'dictionaries': df_dictionaries}

In [7]:
# посмотрим общую информацию о датафреймах

for name, df in dfs.items():
    display(name)
    get_data_info(df)
    print()

'xls'

      id                             Movie   Level
0      0         10_Cloverfield_lane(2016)      B1
1      1  10_things_I_hate_about_you(1999)      B1
2      2              A_knights_tale(2001)      B2
3      3              A_star_is_born(2018)      B2
4      4                     Aladdin(1992)  A2/A2+
..   ...                               ...     ...
236  236                     Matilda(2022)      C1
237  237                      Bullet train      B1
238  238            Thor: love and thunder      B2
239  239                         Lightyear      B2
240  240                        The Grinch      B1

[241 rows x 3 columns]
Общая информация:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      241 non-null    int64 
 1   Movie   241 non-null    object
 2   Level   241 non-null    object
dtypes: int64(1), object(2)
memory usage: 5.8+ KB


None

Первые 5 строк:


,id,Movie,Level
0,0,10_Cloverfield_lane(2016),B1
1,1,10_things_I_hate_about_you(1999),B1
2,2,A_knights_tale(2001),B2
3,3,A_star_is_born(2018),B2
4,4,Aladdin(1992),A2/A2+


...

         word level
0        bank    A1
1         buy    A1
2      course    A1
3         bar    A1
4          by    A1
...       ...   ...
11077     The    C1
11078  Oxford    C1
11079      by    C1
11080    CEFR    C1
11081   level    C1

[11082 rows x 2 columns]
Общая информация:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11082 entries, 0 to 11081
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   word    11082 non-null  object
 1   level   11082 non-null  object
dtypes: object(2)
memory usage: 173.3+ KB


None

Первые 5 строк:


,word,level
0,bank,A1
1,buy,A1
2,course,A1
3,bar,A1
4,by,A1


Количество уникальных значений в каждом столбце:
Столбец: word
Количество уникальных значений: 5032
--------------------
Столбец: level
Количество уникальных значений: 5
--------------------



Дополнительно посмотрим отчеты sweetviz 

In [ ]:
# создадим отчет для df_xls
report_xls = sv.analyze(df_xls)
report_xls.show_html('sweetviz_report_xls.html')

# создадим отчет для df_subtitles
report_subtitles = sv.analyze(df_subtitles)
report_subtitles.show_html('sweetviz_report_subtitles.html')

# создадим отчета для df_dictionaries
report_dictionaries = sv.analyze(df_dictionaries)
report_dictionaries.show_html('sweetviz_report_dictionaries.html')

**Вывод:** 
    
**df_xls:**<br>
Пропущенных значений нет.<br>
В столбце Movie есть дубликаты.<br>
В столбце level есть двойные значения. От них нужно избавиться.<br>

**df_subtitles:**<br> 
Пропущенных значений нет.<br>
Имеются дубликаты.<br>
Необходима предобработка текста в субтитрах.<br> 

**df_dictionaries:**<br> 
Пропущенных значений нет.<br>
Имеются дубликаты в столбце word.<br> 

## Проведем предобработку данных ##

### Проведем предобработку df_xls ### 

In [8]:
# исправим названия столбцов 
df_xls = df_xls.rename(columns={'Movie': 'movie', 'Level': 'level'})

In [9]:
# проверим дубликаты
duplicated_movies = df_xls[df_xls.duplicated('movie', keep=False)]
print(duplicated_movies)

    id                 movie       level
38  38          Powder(1995)          B1
43  43      Inside_out(2015)          B1
44  44      Inside_out(2015)          B1
68  68          Powder(1995)          B1
75  75  The_blind_side(2009)          B2
83  83    The_terminal(2004)          B1
84  84  The_blind_side(2009)          B1
99  99    The_terminal(2004)  A2/A2+, B1


Некоторые фильмы дублируются, но имеют разные уровни. Сохраним более высокий уровень. 

In [10]:
df_xls['level'].value_counts()

B2            101
B1             55
C1             40
A2/A2+         26
B1, B2          8
A2              6
A2/A2+, B1      5
Name: level, dtype: int64

In [11]:
# удалим дубликаты 

# преобразуем уровни в числовые значения
level_mapping = {"A1": 1, "A2": 2, "A2/A2+": 2, "A2/A2+, B1": 3, "B1": 3, "B1, B2": 3, "B2": 4, "C1": 5, "C2": 6}
df_xls['level'] = df_xls['level'].map(level_mapping)

# отсортируем по убыванию) и оставим только первое появление каждого фильма
df_xls = df_xls.sort_values('level', ascending=False).drop_duplicates('movie')

In [12]:
# повторно проверим дубликаты
duplicated_movies = df_xls[df_xls.duplicated('movie', keep=False)]
print(duplicated_movies)

Empty DataFrame
Columns: [id, movie, level]
Index: []


In [13]:
# проверим, какие значения теперь есть в level
df_xls['level'].unique()

array([5, 4, 3, 2])

Нам нужно очистить названия фильмов. Уберем знаки препинания и пробелы. Удалять цифры не будем, так как они могут быть важны, например, если это указание на серию сериала или часть фильма. 

In [14]:
# функция для удаления знаков препинания и пробелов из названий фильмов
def simplify_title(title):
    # Удаляем все символы, кроме букв и цифр
    title = re.sub(r'[^a-zA-Z0-9]', '', title)
    # Преобразуем все к нижнему регистру
    title = title.lower()
    return title

In [15]:
# применим функцию 
df_xls['movie'] = df_xls['movie'].apply(simplify_title)

In [16]:
# проверим результат 
display(df_xls.value_counts())

id   movie                                           level
0    10cloverfieldlane2016                           3        1
167  suitsepisode5toetotoe                           4        1
155  sommintothebottle20151080pblurayx265rarbgensrt  4        1
156  suitsepisode1denial                             4        1
157  suitsepisode10faith                             4        1
                                                             ..
86   thecabininthewoods2012                          2        1
87   thefaultinourstars2014                          3        1
88   thegraduate1967                                 3        1
89   thehangover2009                                 4        1
240  thegrinch                                       3        1
Length: 237, dtype: int64

In [17]:
# посмотрим на финальную версию 
df_xls.head(5)

,id,movie,level
211,211,suitss04e03engsub,5
224,224,suitss04e16engsub,5
64,64,oceanseleven2001,5
221,221,suitss04e13engsub,5
220,220,suitss04e12engsub,5


Удалим столбец id 

In [18]:
df_xls.drop('id', axis=1, inplace=True)

### Проведем предобраобтку df_subtitles ### 

Избавимся от дубликатов

In [19]:
# посмотрим все дублированные строки
df_duplicates = df_subtitles[df_subtitles.duplicated(keep=False)]
print(df_duplicates)
print(df_duplicates.nunique())

                                     movie  \
163                            Dredd(2012)   
165  The_secret_life_of_Walter_Mitty(2013)   
166                            Shrek(2001)   
167                         Deadpool(2016)   
168               We_are_the_Millers(2013)   
..                                     ...   
359             The_greatest_showman(2017)   
360         Kubo_and_the_two_strings(2016)   
361                          Matilda(1996)   
362                              Her(2013)   
363                          Aladdin(1992)   

                                             subtitles  
163  1\n00:00:59,017 --> 00:01:02,103\nAmerica is a...  
165  1\n00:02:33,080 --> 00:02:34,081\nHuh?\n\n2\n0...  
166  1\n00:00:01,000 --> 00:00:04,074\nanoXmous\nht...  
167  1\n00:00:03,259 --> 00:00:20,645\n<font color=...  
168  1\n00:00:02,400 --> 00:00:03,731\n<i>Oh, my Go...  
..                                                 ...  
359  1\n00:00:09,877 --> 00:00:13,081\n♪ Whoa ♪\

In [20]:
# удалим дубликаты строк
df_subtitles.drop_duplicates(inplace=True)

In [21]:
# применим функцию очистки названий к названию фильма
df_subtitles['movie'] = df_subtitles['movie'].apply(simplify_title)

Очистим текст 

In [22]:
# функция для очистки текста 

def clean_text(text):
    # удалим числа и ненужные символы
    cleaned_text = re.sub(r"[^a-z\s']+", " ", text.lower())
    # удалим множественные пробелы
    cleaned_text = re.sub(' +', ' ', cleaned_text)
    # преобразуем строку в список слов
    words = cleaned_text.split()
    return words



In [23]:
# применим функцию к субтитрам
df_subtitles['subtitles'] = df_subtitles['subtitles'].apply(clean_text)

Удалим слова, которые для нас не несут смысловую нагрузку

In [24]:
# загрузим список английских стоп-слов
stop_words = set(stopwords.words('english'))

In [25]:
# функция для удаления знаков пунктуации из списка слов
def remove_punctuation(word_list):
    no_punct = [word for word in word_list if word.isalpha()]
    return no_punct

# функция для удаления стоп-слов из списка слов
def remove_stopwords(word_list):
    no_stop_words = [word for word in word_list if word not in stop_words]
    return no_stop_words

In [26]:
# приведем все слова к нижнему регистру
df_subtitles['subtitles'] = df_subtitles['subtitles'].apply(lambda x: [word.lower() for word in x])


In [27]:
# применим функции 
df_subtitles['subtitles'] = df_subtitles['subtitles'].apply(lambda x: remove_stopwords(x))
df_subtitles['subtitles'] = df_subtitles['subtitles'].apply(lambda x: remove_punctuation(x))

Проведем лемматизацию слов 

In [28]:
# инициализируем лемматизатор
lemmatizer = WordNetLemmatizer()

# функция для лемматизации списка слов
def lemmatize_words(word_list):
    return [lemmatizer.lemmatize(word) for word in word_list]

# применим функцию лемматизации к нашим данным
df_subtitles['subtitles'] = df_subtitles['subtitles'].apply(lemmatize_words)

In [29]:
# cоздадим мешок слов 

# инициализируем CountVectorizer
vectorizer = CountVectorizer()

# объединим слова в строку
df_subtitles['subtitles_str'] = df_subtitles['subtitles'].apply(' '.join)

# применим векторизатор к колонке с субтитрами
bag_of_words = vectorizer.fit_transform(df_subtitles['subtitles_str'])

# преобразуем в numpy array и печатаем результат
bow_matrix = bag_of_words.toarray()
print(bow_matrix)

[[0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 8 0]]


Проведем векторизацию текста

In [30]:
# создадим экземпляр TfidfVectorizer
vectorizer = TfidfVectorizer()

# преобразуем лемматизированные тексты в числовые векторы TF-IDF
tfidf_vectors = vectorizer.fit_transform(df_subtitles['subtitles'].astype(str))

# получим матрицу TF-IDF-векторов
tfidf_matrix = tfidf_vectors.toarray()

# посмотрим результаты
print(tfidf_matrix)

[[0.         0.         0.         ... 0.         0.         0.03307433]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.04324407 0.        ]]


In [31]:
# посмотрим на финальную версию 
df_subtitles.head(5)

,movie,subtitles,subtitles_str
0,crownthes01e01wolfertonsplashen,"[seeking, british, nationalization, royal, hig...",seeking british nationalization royal highness...
1,suitsepisode1denial,"[amazing, woman, ever, met, think, rachel, eli...",amazing woman ever met think rachel elizabeth ...
2,crazy4tvcomsuitss06e06720pblurayx265hevccrazy4ad,"[harvey, reading, sutter, three, year, one, sl...",harvey reading sutter three year one slippery ...
3,suitss02e08hdtvx264evolve,"[car, horn, blare, late, nope, second, early, ...",car horn blare late nope second early good lea...
4,virginrivers01e07internal720pwebx264strife,"[sure, convince, stay, right, time, go, home, ...",sure convince stay right time go home oh know ...


In [32]:
df_subtitles[df_subtitles['movie'] == 'suitss04e03engsub']

,movie,subtitles,subtitles_str
144,suitss04e03engsub,"[previously, suit, good, working, takeover, lo...",previously suit good working takeover logan lo...


### Проведем предобраобтку df_dictionaries ### 

Преобразуем уровни в числовые значения 

In [33]:
# преобразуем уровни в числовые значения
level_mapping = {"A1": 1, "A2": 2, "A2/A2+": 2, "A2/A2+, B1": 3, "B1": 3, "B1, B2": 3, "B2": 4, "C1": 5, "C2": 6}
df_dictionaries["level"] = df_dictionaries["level"].map(level_mapping)


Преобразуем столбец word к нижнему регистру 

In [34]:
# преобразуем столбец "word" к нижнему регистру
df_dictionaries["word"] = df_dictionaries["word"].str.lower()


Проверим дубликаты 

In [35]:
duplicates = df_dictionaries[df_dictionaries.duplicated(keep=False)]
print(duplicates)

         word  level
0        bank      1
1         buy      1
2      course      1
4          by      1
5      cousin      1
...       ...    ...
11077     the      5
11078  oxford      5
11079      by      5
11080    cefr      5
11081   level      5

[8717 rows x 2 columns]


In [36]:
df_dictionaries.value_counts()

word      level
number    1        67
oxford    5        27
          4        25
          2        18
the       4        16
                   ..
indicate  3         1
indirect  2         1
          3         1
indoor    2         1
label     3         1
Length: 6472, dtype: int64

In [37]:
# удалим дубликаты
df_dictionaries = df_dictionaries.drop_duplicates()


In [38]:
df_dictionaries.value_counts()

word         level
a            2        1
precede      4        1
prediction   2        1
predictable  4        1
predict      2        1
                     ..
existence    4        1
exist        2        1
exile        5        1
exhibition   3        1
zone         4        1
Length: 6472, dtype: int64

In [39]:
# посмотрим на финальную версию 
df_dictionaries.sample(5)

,word,level
8882,troubled,5
7701,accountable,5
622,see,1
2100,ingredient,3
2524,prisoner,3


Преобразуем df_dictionaries в словарь 

In [40]:
# преобразуем df_dictionaries в словарь
word_levels = df_dictionaries.set_index('word')['level'].to_dict()


### Объединим таблицы ### 

Объединим df_xls и df_subtitles по столбцу movie 

In [41]:
# объединим таблицы df_xls и df_subtitles по столбцу 'movie'
df_merged = df_xls.merge(df_subtitles, on='movie', how='outer')

display(df_merged.head())


,movie,level,subtitles,subtitles_str
0,suitss04e03engsub,5.0,"[previously, suit, good, working, takeover, lo...",previously suit good working takeover logan lo...
1,suitss04e16engsub,5.0,"[jeff, jessica, right, either, need, going, wo...",jeff jessica right either need going work effe...
2,oceanseleven2001,5.0,"[guard, one, con, escort, open, gate, one, gat...",guard one con escort open gate one gate open m...
3,suitss04e13engsub,5.0,"[got, name, wall, want, respect, come, meaning...",got name wall want respect come meaning want o...
4,suitss04e12engsub,5.0,"[professor, gerard, came, ask, favor, brought,...",professor gerard came ask favor brought know t...


Посмотрим на результат объединения 

In [42]:
get_data_info(df_merged)

                                                 movie  level  \
0                                    suitss04e03engsub    5.0   
1                                    suitss04e16engsub    5.0   
2                                     oceanseleven2001    5.0   
3                                    suitss04e13engsub    5.0   
4                                    suitss04e12engsub    5.0   
..                                                 ...    ...   
281                                     theghostwriter    NaN   
282             harrypotterandthephilosophersstone2001    NaN   
283                                             casper    NaN   
284                                  prideandprejudice    NaN   
285  brenbrownthecalltocourage2019720nf720pddp51x26...    NaN   

                                             subtitles  \
0    [previously, suit, good, working, takeover, lo...   
1    [jeff, jessica, right, either, need, going, wo...   
2    [guard, one, con, escort, open, gate, on

None

Первые 5 строк:


,movie,level,subtitles,subtitles_str
0,suitss04e03engsub,5.0,"[previously, suit, good, working, takeover, lo...",previously suit good working takeover logan lo...
1,suitss04e16engsub,5.0,"[jeff, jessica, right, either, need, going, wo...",jeff jessica right either need going work effe...
2,oceanseleven2001,5.0,"[guard, one, con, escort, open, gate, one, gat...",guard one con escort open gate one gate open m...
3,suitss04e13engsub,5.0,"[got, name, wall, want, respect, come, meaning...",got name wall want respect come meaning want o...
4,suitss04e12engsub,5.0,"[professor, gerard, came, ask, favor, brought,...",professor gerard came ask favor brought know t...


Количество уникальных значений в каждом столбце:
Столбец: movie
Количество уникальных значений: 285
--------------------
Столбец: level
Количество уникальных значений: 4
--------------------
Столбец: subtitles_str
Количество уникальных значений: 278
--------------------


In [43]:
df_merged['level'].value_counts()

4.0    101
3.0     64
5.0     40
2.0     33
Name: level, dtype: int64

***Вывод:*** 
    В получившейся таблице есть пропуски в level и subtitles. 
    Уровень языка мы сможем вычислить, сопоставив слова в субтитрах со словарем. 
    Есть пропуски в subtitles. В идеале можно было бы найти и скачать эти субтитры, но в силу нехватки времени на выполнение проекта удалим эти строки. 
    Итого у нас есть 238 записей для обучения модели (train) и 48 записей для тестирования модели (test). 



Разберемся с пропущенными значениями 

In [44]:
# удалим строки с пропущенными значениями
df_cleaned = df_merged.dropna(subset=['subtitles', 'subtitles_str'])

## Обучим модель ##

In [45]:
# разделим данные на обучающую и тестовую выборки
train = df_cleaned[df_cleaned['level'].notna()]
test = df_cleaned[df_cleaned['level'].isna()]

# разделим данные и train и testabs
x_train = train['subtitles_str']
y_train = train['level']
x_test = test['subtitles_str']

Применим мешок слов 

In [46]:
# применим мешка слов
vectorizer = CountVectorizer()
x_train_counts = vectorizer.fit_transform(x_train)
x_test_counts = vectorizer.transform(x_test)


Применим tfidf

In [47]:
# применим TF-IDF
tfidf_transformer = TfidfVectorizer()
x_train_tfidf = tfidf_transformer.fit_transform(x_train)
x_test_tfidf = tfidf_transformer.transform(x_test)


Сбалансируем классы 

In [48]:
# применим балансировку классов 
ros = RandomOverSampler(random_state=0)
x_resampled, y_resampled = ros.fit_resample(x_train_tfidf, y_train)

Обучим модель логистической регрессии

In [49]:
# обучим модель логистической регрессии с подбором гиперпараметров и кросс-валидацией
logistic_regression = LogisticRegression(max_iter=1000)
param_grid_lr = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l2']
}
grid_search_lr = GridSearchCV(estimator=logistic_regression, param_grid=param_grid_lr, cv=5, scoring='accuracy')
grid_search_lr.fit(x_resampled, y_resampled)

best_model_lr = grid_search_lr.best_estimator_

# оценка качества модели на кросс-валидации
cv_results_lr = cross_val_score(best_model_lr, x_resampled, y_resampled, cv=5, scoring='accuracy')

print('Лучшие параметры логистической регрессии: ', grid_search_lr.best_params_)
print('Точность логистической регрессии на кросс-валидации: ', cv_results_lr.mean())

Лучшие параметры логистической регрессии:  {'C': 100, 'penalty': 'l2'}
Точность логистической регрессии на кросс-валидации:  0.860939060939061


Обучим модель градиентного бустинга

In [52]:
# обучим модель градиентного бустинга с подбором гиперпараметров и кросс-валидацией
gradient_boosting = GradientBoostingClassifier()
param_grid_gb = {
    'n_estimators': [50, 100],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5]
}
grid_search_gb = GridSearchCV(estimator=gradient_boosting, param_grid=param_grid_gb, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_gb.fit(x_resampled, y_resampled)

best_model_gb = grid_search_gb.best_estimator_

# оценим качество модели на кросс-валидации
cv_results_gb = cross_val_score(best_model_gb, x_resampled, y_resampled, cv=5, scoring='accuracy')

print('Лучшие параметры градиентного бустинга: ', grid_search_gb.best_params_)
print('Точность градиентного бустинга на кросс-валидации: ', cv_results_gb.mean())

Лучшие параметры градиентного бустинга:  {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
Точность градиентного бустинга на кросс-валидации:  0.8789876789876789


***Вывод:*** 

Лучшие параметры логистической регрессии:  {'C': 100, 'penalty': 'l2'}
Точность логистической регрессии на кросс-валидации:  0.860939060939061

Лучшие параметры градиентного бустинга:  {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
Точность градиентного бустинга на кросс-валидации:  0.8789876789876789

Модель градиентного бустинга показала немного более высокий результат. 

***Итоговый вывод:*** 
Мы построили две модели для предсказания уровня сложности фильма по его субтитрам: модель логистической регрессии и модель градиентного бустинга.<br> 

Лучшие параметры логистической регрессии:  {'C': 100, 'penalty': 'l2'}
Точность логистической регрессии на кросс-валидации:  0.860939060939061

Лучшие параметры градиентного бустинга:  {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
Точность градиентного бустинга на кросс-валидации:  0.8789876789876789

Более высокий результат показала модель градиентного бустинга со следующими гиперпараметрами: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}. 
Точность градиентного бустинга на кросс-валидации составила примерно 0.879, что означает, что модель с лучшими найденными параметрами достигает примерно 87.9% точности в предсказании уровня английского языка.